In [1]:
import azureml
from azureml.core import Workspace, Run, Experiment
from azureml.train.estimator import Estimator

import os, shutil
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import datetime as dt
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

In [2]:
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version: 1.0.53

In [3]:
# set aml workspace parameters here. 
subscription_id = ""
resource_group = ""
workspace_name = ""
workspace_region = ""

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)


In [4]:
# create experiment
experiment_name = 'bikeSharingDemand'
exp = Experiment(workspace=ws, name=experiment_name)

In [5]:
run = exp.start_logging(snapshot_directory=None)

In [6]:
df = (spark
        .read
        .format("csv")
        .option("inferSchema", "True")
        .option("header", "True")
        .load("/databricks-datasets/bikeSharing/data-001/day.csv")
       )

# split data
train_df, test_df = df.randomSplit([0.7, 0.3])

# One Hot Encoding
mnth_encoder = OneHotEncoder(inputCol="mnth", outputCol="encoded_mnth")
weekday_encoder = OneHotEncoder(inputCol="weekday", outputCol="encoded_weekday")

# set the training variables we want to use
train_cols = ['encoded_mnth', 'encoded_weekday', 'temp', 'hum']

# convert cols to a single features col
assembler = VectorAssembler(inputCols=train_cols, outputCol="features")

# Set linear regression model
lr = LinearRegression(featuresCol="features", labelCol="cnt")

# Create pipeline
pipeline = Pipeline(stages=[
    mnth_encoder,
    weekday_encoder,
    assembler,
    lr
])

# fit pipeline
lrPipelineModel = pipeline.fit(train_df)

# write test predictions to datetime and lastest folder
predictions = lrPipelineModel.transform(test_df)

# mlflow log evaluations
evaluator = RegressionEvaluator(labelCol = "cnt", predictionCol = "prediction")

run.log("mae", evaluator.evaluate(predictions, {evaluator.metricName: "mae"}))
run.log("rmse", evaluator.evaluate(predictions, {evaluator.metricName: "rmse"}))
run.log("r2", evaluator.evaluate(predictions, {evaluator.metricName: "r2"}))


In [7]:
model_nm = "bikeshare.mml"
model_output = '/mnt/azml/outputs/'+model_nm
model_dbfs = "/dbfs"+model_output
lrPipelineModel.write().overwrite().save(model_output)

In [8]:
model_name, model_ext = model_dbfs.split(".")

In [9]:
model_zip = model_name + ".zip"
shutil.make_archive(model_name, 'zip', model_dbfs)
run.upload_file("outputs/" + model_nm, model_zip)

Out[ 35 ]: <azureml._restclient.models.batch_artifact_content_information_dto.BatchArtifactContentInformationDto at 0x7fe3b8ae6eb8>

In [10]:
# looks to register the model we published to the output folder of our workspace
run.register_model(model_name = 'model_nm', model_path = "outputs/" + model_nm)


Out[ 37 ]: Model(workspace=Workspace.create(name='demoworkspace', subscription_id='4ad547a3-bf5b-454a-925c-56e663a6d6e4', resource_group='AML'), name=model_nm, id=model_nm:1, version=1, tags={}, properties={})

In [11]:
# now delete the serialized model from local folder since it is already uploaded to run history 
shutil.rmtree(model_dbfs)
os.remove(model_zip)

In [12]:
run.complete()